
# SixtyFour AI — QA Agent (Beta)

This notebook is your guide to using the Sixtyfour API `qa-agent` (beta).

The `qa-agent` endpoint evaluates and qualifies data against predefined criteria using autonomous research and structured analysis. It can read your provided data, perform targeted research (optionally using your references), score against your criteria, and return a clean `structured_data` block plus notes.

**Tip:** You can change the `data` object and criteria at the top. Make criteria **clear and weighted** for best results.



## Prerequisites & Secrets

### Get your SixtyFour API key
1) Sign in: https://app.sixtyfour.ai/login  
2) Create an organization → **Keys** → **Create new key** → Copy the key.

### If you’re in Google Colab
- Click the 🔑 (Secrets) icon in the left sidebar.  
- Add `SIXTYFOUR_API_KEY` and paste your key.  
- Make sure the notebook can access the secret.


In [2]:

# --- Parameters (you can change these) ---
DATA_NAME = "Saarth Shah"
DATA_BUSINESS_TYPE = "Bakery"
DATA_FOLLOWERS = "1 million"
DATA_TWITTER = "saarth_"
DATA_POSTS = "6500"

import os, json, requests

API_KEY = os.getenv("SIXTYFOUR_API_KEY", "")
try:
    from google.colab import userdata
    secret_val = userdata.get("SIXTYFOUR_API_KEY")
    if secret_val:
        os.environ["SIXTYFOUR_API_KEY"] = secret_val
        API_KEY = secret_val
except Exception:
    pass

BASE_URL = "https://api.sixtyfour.ai"
ENDPOINT = f"{BASE_URL}/qa-agent"
TIMEOUT_S = 600

print("✅ Setup complete (key detected, endpoint ready).")


✅ Setup complete (key detected, endpoint ready).



## QA_Agent Request

For most use cases, `qa-agent` is simple: pass a `data` object, a list of `qualification_criteria`, and (optionally) `references` and `struct`.

The request body requires:
- **data (object)**: Primary data to evaluate and research.
- **qualification_criteria (array)**: List of criteria with names, descriptions, weights, and optional thresholds.

**Optional fields:**
- **references (array)**: URLs to guide research (LinkedIn, Instagram, websites).  
- **struct (object)**: Output fields to include in `structured_data` (names + descriptions).


In [8]:
data_obj = {
    "name": DATA_NAME,
    "business_type": DATA_BUSINESS_TYPE,
    "follower_count": DATA_FOLLOWERS,
    "post_count": DATA_POSTS,
    "twitter_handle": DATA_POSTS,
}

criteria = [
    {
        "criteria_name": "Follower count",
        "description": "Must have at least 2,500 followers",
        "weight": 10.0,
        "threshold": 8.0
    },
    {
        "criteria_name": "Target Market Location",
        "description": "Must be located in San Francisco",
        "weight": 8.0
    }
]

references = [
    {"url": f"https://www.x.com/{DATA_TWITTER}", "description": "Twitter profile"}
]

# You may optionally specify types for predictable outputs inside struct fields (e.g., {"type": "int"}).
struct = {
    "suitability_score": "Overall qualification score (0-10)",
    "disqualification_reason": "Primary reason for disqualification if applicable",
    "final_verdict": "Clear YES/NO decision",
    "recommendation": "ACCEPT/REJECT with reasoning"
}

payload = {
    "data": data_obj,
    "qualification_criteria": criteria,
    "references": references,
    "struct": struct
}

resp = requests.post(
    ENDPOINT,
    headers={"x-api-key": API_KEY, "Content-Type": "application/json"},
    json=payload,
    timeout=TIMEOUT_S,
)

if resp.ok:
    result = resp.json()
    print("✅ Success! Keys:", list(result.keys()))
    print("\nStructured data:")
    print(json.dumps(result.get("structured_data", {}), indent=2))
    print("\nNotes:")
    print(result.get("notes", ""))
else:
    print("❌ Request failed:", resp.status_code, resp.text[:800])

✅ Success! Keys: ['structured_data', 'notes']

Structured data:
{
  "recommendation": "REJECT due to follower count below required threshold and insufficient evidence of bakery business in San Francisco",
  "final_verdict": "NO",
  "disqualification_reason": "Follower count below minimum requirement of 2,500; no verified bakery business presence in San Francisco",
  "suitability_score": "3"
}

Notes:
Research findings indicate that Saarth Shah is primarily a tech entrepreneur and software engineer based in San Francisco, involved with a company named Sixtyfour, not explicitly a bakery owner. The Twitter profile (https://www.x.com/saarth_) shows about 2,640 followers, barely above the 2,500 threshold, but the original data claiming 1 million followers and post count '6500' is inconsistent and unverified. No official bakery website or social media presence related to a bakery business was found despite multiple targeted searches and attempts to scrape potential bakery sites, including ht


### How to change values

- Edit fields in `data_obj` for your candidate/business.  
- Adjust `criteria` names, descriptions, weights, and thresholds to match your rubric.  
- Add `references` (LinkedIn, Instagram, websites) to guide research.  
- Modify `struct` to control which fields appear in `structured_data`.



## Example criteria & struct (copy/paste)


In [ ]:

example_criteria_light = [
    {"criteria_name": "Active Posting", "description": "Posts at least weekly", "weight": 5.0},
    {"criteria_name": "Audience Size", "description": "At least 1000 followers", "weight": 3.0}
]

example_criteria_strict = [
    {"criteria_name": "Inventory Ownership", "description": "Owns and ships products directly", "weight": 10.0, "threshold": 9.0},
    {"criteria_name": "EU Location", "description": "Located in EU", "weight": 6.0}
]

example_struct_min = {
    "suitability_score": "Overall score (0-10)",
    "final_verdict": "YES/NO decision"
}

print("Ready-to-use examples:")
print("criteria (light):", example_criteria_light)
print("criteria (strict):", example_criteria_strict)
print("struct (minimal):", example_struct_min)
